In [1]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph import MessagesState, StateGraph, START, END
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage
from typing import Literal, TypedDict
import sympy as sp
import re

class AgentState(TypedDict):
    messages: list
    plan: str
    current_step: int

llm = ChatOpenAI(model='gpt-4o-mini-2024-07-18', temperature=0)

# Define a single sympy-based tool
@tool
def calculate(expression: str) -> str:
    """Calculate an arithmetic expression using sympy.

    Args:
        expression: arithmetic expression as a string (e.g., '3 + 4')

    Returns:
        The evaluated result as a string.
    """
    try:
        expr = sp.sympify(expression)
        evaluated = expr.evalf() if expr.is_number else expr
        return str(evaluated)
    except Exception as e:
        return f"Error: {e}"
    

@tool
def format_answer(text: str) -> str:
    """Returns only the numerical result (no text)."""
    numbers = re.findall(r"[-+]?\d*\.\d+|\d+", text)
    if numbers:
        return numbers[-1].split('.')[0] if numbers[-1].endswith('.0') else numbers[-1]
    return text



# Augment the LLM with the sympy tool
tools = [calculate, format_answer]
tools_by_name = {tool.name: tool for tool in tools}

# Planner node
def planner(state: AgentState):
    """Generate a step-by-step plan to solve the problem"""
    system_message = SystemMessage(
        content="""You are a math problem solving planner. Analyze the problem and create a step-by-step plan to solve it.
        Include which tools to use and in what order. Available tools:
        - calculate: for math expressions (e.g., '3 + 4')
        - format_answer: to clean numerical results
        
        Return ONLY the plan as a numbered list.
        Example plan:
        1. Calculate food cost: 30% of $1000 using calculate tool
        2. Calculate accommodation cost: 15% of $1000 using calculate tool
        3. Sum food and accommodation costs using calculate tool
        4. Subtract from total budget using calculate tool
        5. Format final answer using format_answer tool"""
    )
    
    response = llm.invoke([system_message] + state["messages"])
    return {"messages": state["messages"] + [response], "plan": response.content}

# Executor node
def executor(state: AgentState):
    """Execute the current step of the plan using your original tools"""
    if not state.get("plan"):
        return state
    
    # Parse current step from plan
    steps = [step for step in state["plan"].split('\n') if step.strip()]
    current_step = state.get("current_step", 0)
    
    if current_step >= len(steps):
        return state
    
    system_message = SystemMessage(
        content=f"Execute this step: {steps[current_step]}\n"
                "Use tools when needed and be concise."
    )
    
    llm_with_tools = llm.bind_tools(tools)
    response = llm_with_tools.invoke([system_message] + state["messages"])
    
    # Execute tools exactly as in your original implementation
    result = []
    if response.tool_calls:
        for tool_call in response.tool_calls:
            tool = tools_by_name[tool_call["name"]]
            observation = tool.invoke(tool_call["args"])
            result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
    
    return {
        "messages": state["messages"] + [response] + result,
        "plan": state["plan"],
        "current_step": current_step + 1
    }

# Final response node (using your format_answer tool)
def final_response(state: AgentState):
    """Generate the final answer in exact 'Answer: <number>' format"""
    # First get the raw numerical result from previous steps
    numerical_result = None
    for msg in reversed(state["messages"]):
        if hasattr(msg, 'content') and re.search(r'\d+', msg.content):
            numerical_result = msg.content
            break
    
    # Use format_answer tool to clean the number
    formatted_number = format_answer.invoke(numerical_result) if numerical_result else "0"
    
    # Create the exact output format
    system_message = SystemMessage(
        content="You must format the final answer as 'Answer: <number>' with no additional text. "
                "Example: 'Answer: 42'"
    )
    
    final_output = llm.invoke([
        system_message,
        HumanMessage(content=f"The number to format is: {formatted_number}")
    ])
    
    return {
        "messages": state["messages"] + [final_output],
        "plan": state["plan"],
        "current_step": state["current_step"]
    }

# Conditional edges
def should_continue(state: AgentState) -> Literal["execute", "finalize", END]:
    steps = [step for step in state["plan"].split('\n') if step.strip()]
    current_step = state.get("current_step", 0)
    
    if current_step < len(steps):
        return "execute"
    return "finalize"


workflow = StateGraph(AgentState)
workflow.add_node("planner", planner)
workflow.add_node("executor", executor)
workflow.add_node("final_response", final_response)

# Set edges
workflow.add_edge("planner", "executor")
workflow.add_conditional_edges(
    "executor",
    should_continue,
    {
        "execute": "executor",
        "finalize": "final_response"
    }
)
workflow.add_edge("final_response", END)
workflow.set_entry_point("planner")

# Compile the agent
agent = workflow.compile()

def get_final_answer(result):
    for m in reversed(result["messages"]):
        if hasattr(m, 'content') and m.content:
            match = re.search(r'Answer:\s*([\d]+)', m.content)
            if match:
                return f"Answer: {match.group(1)}"
    return "No answer found in expected format"

# Example usage 
messages = [HumanMessage(
    content="James has a $1000 budget. He spends 30% on food, "
            "15% on accommodation, 25% on entertainment. "
            "How much does he spend on coursework materials?"
)]

result = agent.invoke({"messages": messages})
print(get_final_answer(result)) 

Answer: 300
